In [2]:
#importamos todas las librerias necesarias para el proyecto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, csv, ast

In [3]:
#cargamos el archivo csv
df_credits=pd.read_csv('credits.csv')

#imprimimos y vemos su dimension
print(df_credits.head(5))
df_credits.shape

                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862  


(45476, 3)

In [4]:
#desanidamos cast
def desanidar_cast(cadena_json):
    try:
        lista_actores = json.loads(cadena_json.replace("'", '"')) #replace single quotes with double quotes
        actores = [{'cast_id': actor.get('cast_id'), 'character': actor.get('character'), 'name': actor.get('name')} for actor in lista_actores]
        return actores
    except (TypeError, json.JSONDecodeError, KeyError, AttributeError):
        return None

df_credits['cast'] = df_credits['cast'].apply(desanidar_cast)

In [5]:
#desanidamos crew
def desanidar_crew(cadena_json):
    try:
        lista_crew = json.loads(cadena_json.replace("'", '"')) #replace single quotes with double quotes
        crew = [{'credit_id': miembro.get('credit_id'), 'department': miembro.get('department'), 'job': miembro.get('job'), 'name': miembro.get('name')} for miembro in lista_crew]
        return crew
    except (TypeError, json.JSONDecodeError, KeyError, AttributeError):
        return None

df_credits['crew'] = df_credits['crew'].apply(desanidar_crew)

In [6]:
#expandimos las listas
df_credits = df_credits.explode('cast')
df_credits = df_credits.explode('crew')

In [12]:
#reiniciamos índices
df_credits = df_credits.reset_index(drop=True)
df_cast = df_cast.reset_index(drop=True)
df_crew = df_crew.reset_index(drop=True)

#cncatenamos DataFrames
df_credits = pd.concat([df_credits, df_cast, df_crew], axis=1)

#eliminamos columnas originales
df_credits = df_credits.drop(['cast', 'crew'], axis=1)

In [14]:
#eliminamos las filas con valor NaN
df_credits = df_credits.dropna()

#convertimos la columna 'cast_id' a enteros
df_credits['cast_id'] = df_credits['cast_id'].astype(int)

In [16]:
print(df_credits.head(10))


       id  cast_id  character                name                 credit_id  \
58  16420        2    Othello  Laurence Fishburne  5462510ec3a368082d000121   
59  16420        2    Othello  Laurence Fishburne  52fe46d29251416c75084b07   
60  16420        2    Othello  Laurence Fishburne  54625118c3a368081300010e   
61  16420        3  Desdemona         Irène Jacob  5462510ec3a368082d000121   
62  16420        3  Desdemona         Irène Jacob  52fe46d29251416c75084b07   
63  16420        3  Desdemona         Irène Jacob  54625118c3a368081300010e   
64  16420        4       Iago     Kenneth Branagh  5462510ec3a368082d000121   
65  16420        4       Iago     Kenneth Branagh  52fe46d29251416c75084b07   
66  16420        4       Iago     Kenneth Branagh  54625118c3a368081300010e   
67  16420        5     Cassio    Nathaniel Parker  5462510ec3a368082d000121   

   department         job                 name  
58    Writing      Writer  William Shakespeare  
59  Directing    Director       

In [17]:
df_credits.to_csv('credits_modificado.csv', index=False)